# Classification on CIFAR and ImageNet

In [3]:
%load_ext autoreload
%autoreload 2

import sys

# check whether run in Colab
root = "."
if "google.colab" in sys.modules:
    print("Running in Colab.")
    !pip3 install matplotlib
    !pip3 install einops==0.3.0
    !pip3 install timm==0.4.9
    !git clone -b transformer git@github.com:JacobARose/how-do-vits-work.git
    root = "./how-do-vits-work"
    sys.path.append(root)

In [4]:
import os
import time
import yaml
import copy
from pathlib import Path
import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import models
import ops.trains as trains
import ops.tests as tests
import ops.datasets as datasets
import ops.schedulers as schedulers

In [5]:
# config_path = "%s/configs/cifar10_vit.yaml" % root
config_path = "%s/configs/cifar100_vit.yaml" % root
# config_path = "%s/configs/imagenet_vit.yaml" % root

with open(config_path) as f:
    args = yaml.load(f)
    print(args)

{'dataset': {'name': 'cifar100', 'root': '../data', 'mean': [0.5071, 0.4867, 0.4408], 'std': [0.2675, 0.2565, 0.2761], 'padding': 4, 'color_jitter': 0.0, 'auto_augment': 'rand-m9-n2-mstd1.0', 're_prob': 0.0}, 'train': {'warmup_epochs': 5, 'epochs': 300, 'batch_size': 96, 'max_norm': 5, 'smoothing': 0.1, 'mixup': {'num_classes': 100, 'mixup_alpha': 1.0, 'cutmix_alpha': 0.8, 'prob': 1.0}}, 'val': {'batch_size': 256, 'n_ff': 1}, 'model': {'stem': False, 'block': {'image_size': 32, 'patch_size': 2, 'sd': 0.1}}, 'optim': {'name': 'AdamW', 'lr': 0.000125, 'weight_decay': 0.05, 'scheduler': {'name': 'CosineAnnealingLR', 'T_max': 300, 'eta_min': 0}}, 'env': {}}


/tmp/ipykernel_3748349/3079580007.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  args = yaml.load(f)


In [6]:
dataset_args = copy.deepcopy(args).get("dataset")
train_args = copy.deepcopy(args).get("train")
val_args = copy.deepcopy(args).get("val")
model_args = copy.deepcopy(args).get("model")
optim_args = copy.deepcopy(args).get("optim")
env_args = copy.deepcopy(args).get("env")

In [7]:
dataset_train, dataset_test = datasets.get_dataset(**dataset_args, download=True)
dataset_name = dataset_args["name"]
num_classes = len(dataset_train.classes)

dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=train_args.get("num_workers", 4), 
                           batch_size=train_args.get("batch_size", 128))
dataset_test = DataLoader(dataset_test, 
                          num_workers=val_args.get("num_workers", 4), 
                          batch_size=val_args.get("batch_size", 128))

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    num_classes
))

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ../data/cifar-100-python.tar.gz to ../data
Files already downloaded and verified
Train: 50000, Test: 10000, Classes: 100


## Model

Use provided models:

In [ ]:
# ResNet
# name = "resnet_dnn_50"
# name = "resnet_dnn_101"

# ViT
name = "vit_ti"
# name = "vit_s"

vit_kwargs = {  # for CIFAR
    "image_size": 32, 
    "patch_size": 2,
}

model = models.get_model(name, num_classes=num_classes, 
                         stem=model_args.get("stem", False), **vit_kwargs)
# models.load(model, dataset_name, uid=current_time)

# Or use `timm`:

import timm

model = timm.models.vision_transformer.VisionTransformer(
    img_size=32, patch_size=2, num_classes=num_classes,  # for CIFAR
    embed_dim=192, depth=12, num_heads=3, qkv_bias=False,  # ViT-Ti
)
model.name = "vit_ti"
models.stats(model)

Parallelize the given `moodel` by splitting the input:

In [12]:
name = model.name
model = nn.DataParallel(model)
model.name = name

## Train

Define a TensorBoard writer:

In [13]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

with open("%s/config.yaml" % log_dir, "w") as f:
    yaml.dump(args, f)
with open("%s/model.log" % log_dir, "w") as f:
    f.write(repr(model))

print("Create TensorBoard log dir: ", log_dir)

Create TensorBoard log dir:  runs/cifar100/vit_ti/20220516_062744


Train the model:

In [ ]:
gpu = torch.cuda.is_available()
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * train_args.get("warmup_epochs", 0))

trains.train(model, optimizer,
             dataset_train, dataset_test,
             train_scheduler, warmup_scheduler,
             train_args, val_args, gpu,
             writer, 
             snapshot=-1, 
             dataset_name=dataset_name, 
             uid=current_time)  # Set `snapshot=N` to save snapshots every N epochs.

NLL: 2.1916, Cutoffs: 0.0 %, 90.0 %, Accs: 47.660 %, 100.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 31.472 %, 100.000 %, Freqs: 100.000 %, 0.190 %, Top-5: 78.010 %, Brier: 0.704, ECE: 18.315 %, ECE±: -18.315 %


Batches-per-epoch=521:   0%|          | 0/521 [00:00<?, ?it/s]

NLL: 2.1385, Cutoffs: 0.0 %, 90.0 %, Accs: 48.650 %, 100.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 31.934 %, 100.000 %, Freqs: 100.000 %, 0.310 %, Top-5: 78.020 %, Brier: 0.690, ECE: 17.515 %, ECE±: -17.515 %


Batches-per-epoch=521:   0%|          | 0/521 [00:00<?, ?it/s]

NLL: 2.1342, Cutoffs: 0.0 %, 90.0 %, Accs: 49.220 %, 100.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 32.963 %, 100.000 %, Freqs: 100.000 %, 0.170 %, Top-5: 78.330 %, Brier: 0.689, ECE: 18.316 %, ECE±: -18.316 %


...

Batches-per-epoch=521:   0%|          | 0/521 [00:00<?, ?it/s]

NLL: 1.4578, Cutoffs: 0.0 %, 90.0 %, Accs: 66.000 %, 100.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 50.194 %, 100.000 %, Freqs: 100.000 %, 2.160 %, Top-5: 87.780 %, Brier: 0.475, ECE: 12.524 %, ECE±: -12.524 %


Batches-per-epoch=521:   0%|          | 0/521 [00:00<?, ?it/s]

NLL: 1.4738, Cutoffs: 0.0 %, 90.0 %, Accs: 65.590 %, 98.165 %, Uncs: 0.000 %, 99.884 %, IoUs: 49.713 %, 96.786 %, Freqs: 100.000 %, 2.180 %, Top-5: 87.710 %, Brier: 0.478, ECE: 12.176 %, ECE±: -12.176 %


Batches-per-epoch=521:   0%|          | 0/521 [00:00<?, ?it/s]

Save the model:

In [17]:
models.save(model, dataset_name, current_time, optimizer=optimizer)

## Test

In [18]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s.csv" % (dataset_name, model.name, current_time))
tests.save_metrics(metrics_dir, metrics_list)

N: 1, NLL: 1.4183, Cutoffs: 0.0 %, 90.0 %, Accs: 67.240 %, 96.992 %, Uncs: 0.000 %, 99.878 %, IoUs: 51.552 %, 94.262 %, Freqs: 100.000 %, 1.330 %, Top-5: 88.040 %, Brier: 0.458, ECE: 12.532 %, ECE±: -12.532 %


In [16]:
85/15

5.666666666666667